In [5]:
import cv2
import dlib
import numpy as np
import winsound
from scipy.spatial import distance

In [6]:
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])  # vertical
    B = distance.euclidean(eye[2], eye[4])  # vertical
    C = distance.euclidean(eye[0], eye[3])  # horizontal
    ear = (A + B) / (2.0 * C)
    return ear

In [7]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

(lStart, lEnd) = (42, 48)  # right eye
(rStart, rEnd) = (36, 42)  # left eye

In [8]:
cap = cv2.VideoCapture(0)
cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)

DROWSY_EAR_THRESHOLD = 0.25
DROWSY_FRAMES = 20
counter = 0

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray, 0)
    for face in faces:
        shape = predictor(gray, face)
        shape_np = np.zeros((68, 2), dtype="int")

        for i in range(0, 68):
            shape_np[i] = (shape.part(i).x, shape.part(i).y)

        leftEye = shape_np[lStart:lEnd]
        rightEye = shape_np[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        ear = (leftEAR + rightEAR) / 2.0

        if ear < DROWSY_EAR_THRESHOLD:
            counter += 1
            if counter >= DROWSY_FRAMES:
                print("Drowsiness detected!")
                print("EAR Value:", ear)
                cv2.putText(frame, "DROWSINESS ALERT!", (50, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
    
    # Windows sound alert
                winsound.Beep(1000, 1000)  # frequency, duration (in ms)
        else:
            counter = 0

        for (x, y) in np.concatenate((leftEye, rightEye), axis=0):
            cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()

Drowsiness detected!
EAR Value: 0.17285034168930827
Drowsiness detected!
EAR Value: 0.18305794458395536
Drowsiness detected!
EAR Value: 0.18380223048358624
Drowsiness detected!
EAR Value: 0.16788072768718704
Drowsiness detected!
EAR Value: 0.17086001917838115
Drowsiness detected!
EAR Value: 0.17271351290748432
Drowsiness detected!
EAR Value: 0.19787978871071862
